In [ ]:
import sys, os, importlib
sys.path.append(os.path.abspath('../'))
from wmhpypes.interfaces import ibbmTum

In [ ]:
import importlib
importlib.reload(ibbmTum)

In [ ]:
os.getcwd()

In [ ]:
test_data_dir = os.path.abspath('../test_data')
weights_dir = os.path.join(test_data_dir, 'model_FLAIR_only')
out_dir = os.path.join(test_data_dir, 'results_FLAIR_only')
wf_work_dir = os.path.join(test_data_dir, 'wf_work_dir')

In [ ]:
flair = '/media/orco/home_data/Dokumente/DZNE/WMHpypes/test_data/sub-test/sub-test_FLAIR.nii.gz'
weights = [os.path.abspath(os.path.join(weights_dir, i)) for i in os.listdir(weights_dir)]

In [ ]:
os.path.isfile(weights[0])

In [ ]:
# Preprocessing

In [ ]:
p = ibbmTum.Preprocessing()

In [ ]:
p.inputs.cols_standard=200
p.inputs.rows_standard=200
p.inputs.flair = flair
p.inputs.thres = 30

In [ ]:
res = p.run()

In [ ]:
res.outputs

In [ ]:
importlib.reload(ibbmTum)

In [ ]:
# Predict
import numpy as np
predict_input = np.load('./preprocessed.npy')

In [ ]:
# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1,
#                                                    inter_op_parallelism_threads=1)))

In [ ]:
p = ibbmTum.Predict()

In [ ]:
p.inputs.slice_shape = (200,200,1)
p.inputs.preprocessed_array = predict_input
p.inputs.weights = weights

In [ ]:
predres = p.run()

In [ ]:
#res.outputs.prediction

In [ ]:
# Save prediction

In [ ]:
sp = ibbmTum.SavePrediction()
sp.inputs.prediction_array = predres.outputs.prediction
spres = sp.run()

In [ ]:
spres.outputs.prediction_nifti

In [ ]:
wmh_nii = '/home/orco/Documents/DZNE/WMHpypes/test_data/results_FLAIR_only/_sub-id_sub-test/prediction.nii.gz'

In [ ]:
import nibabel as nib
img = nib.load(wmh_nii)

In [ ]:
img.affine

In [ ]:
# Extract affine

In [ ]:
from wmhpypes.interfaces import misc

In [ ]:
import importlib
importlib.reload(misc)
ExtractAffine = misc.ExtractAffine

In [ ]:
ea = ExtractAffine()

In [ ]:
ea.inputs.in_nii = '/media/orco/home_data/Dokumente/DZNE/WMHpypes/test_data/sub-test/sub-test_FLAIR.nii.gz'
#ea.inputs.output_filename = 'affine.mat'

In [ ]:
eares = ea.run()

In [ ]:
eares.outputs.out_matrix

In [ ]:
eares.outputs.out_file

In [ ]:
# Save NIfTI

In [ ]:
import importlib
importlib.reload(misc)
SaveNIfTI = misc.SaveNIfTI

In [ ]:
sn = SaveNIfTI()
sn.inputs.in_array = wmh_nii
sn.inputs.in_header = '/media/orco/home_data/Dokumente/DZNE/WMHpypes/test_data/sub-test/sub-test_FLAIR.nii.gz'
sn.inputs.in_matrix = np.loadtxt('affine.mat')
sn.inputs.out_filename = 'wmh_mask'
snres = sn.run()

In [ ]:
import nilearn.plotting as plotting

In [ ]:
nib.load('./wmh_mask.nii.gz')

In [ ]:
plotting.plot_roi('./wmh_mask.nii.gz', '/media/orco/home_data/Dokumente/DZNE/WMHpypes/test_data/sub-test/sub-test_FLAIR.nii.gz')

In [ ]:
plotting.plot_roi('./prediction.nii.gz', '/media/orco/home_data/Dokumente/DZNE/WMHpypes/test_data/sub-test/sub-test_FLAIR.nii.gz')

In [ ]:
# CAT12 denoising

In [ ]:
from wmhpypes.interfaces import cat12
import importlib
importlib.reload(cat12)
CAT12SANLMDenoising = cat12.CAT12SANLMDenoising

In [ ]:
c = CAT12SANLMDenoising()

In [ ]:
c.inputs.in_files='sub-test_FLAIR.nii'
c.inputs.rician = 0

In [ ]:
res = c.run()

In [ ]:
res

In [ ]:
import sys, os
sys.path.append(os.path.abspath('../'))

from wmhpypes.workflows import ibbmTum_wf
from nipype.pipeline.engine import Workflow, Node

from nipype import IdentityInterface
test_wf = ibbmTum_wf.get_test_wf()

In [ ]:
flair = Node(interface=IdentityInterface(fields=['flair']), name='flair')
flair.inputs.flair = os.path.abspath('./sub-test_FLAIR.nii')
weights = Node(interface=IdentityInterface(fields=['weights']), name='weights')
weights.inputs.weights = [os.path.abspath('./0.h5')]
sink = Node(interface=DataSink(), name ='sink')
sink.inputs.base_directory = './out'

wmh = Workflow(name='wmh', base_dir='./wf')
wmh.connect(weights, 'weights', test_wf, 'inputspec.weights')
wmh.connect(flair, 'flair', test_wf, 'inputspec.flair')

In [ ]:
wmh.run()

In [ ]:
wmh.inputs

In [ ]:
wmh.outputs

In [6]:
import sys, os, importlib
sys.path.append(os.path.abspath('../'))
from wmhpypes.interfaces import misc
import importlib
importlib.reload(misc)

<module 'wmhpypes.interfaces.misc' from '/home/dellorcoa/anaconda3/envs/wmhpypes/lib/python3.9/site-packages/wmhpypes/interfaces/misc.py'>

In [7]:
s = misc.SaveNIfTI()

In [8]:
s.inputs.in_array = './prediction.nii'
s.inputs.in_header = './sub-test_FLAIR.nii.gz'
s.inputs.in_matrix = './sub-test_FLAIR.nii.gz'
s.inputs.out_filename = 'pred_mod.nii.gz'

In [9]:
s.run()

sizeof_hdr should be 540; set sizeof_hdr to 540
